# 第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

### 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [3]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1


with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

morphs = []
for i in range(len(lines)):
    if lines[i] == "" or lines[i][0] == "*":
       continue
    if lines[i] == 'EOS':
        morphs.append(Morph('EOS', 'EOS', 'EOS', 'EOS'))
    else:
        sur, keys = lines[i].split("\t")
        key = keys.split(",")
        morphs.append(Morph(sur, key[6], key[0], key[1]))

for i in range(1, len(morphs)):
    if morphs[i].surface == 'EOS':
        break
    print(f"surface: {morphs[i].surface}, base: {morphs[i].base}, pos: {morphs[i].pos}, pos1: {morphs[i].pos1}")

surface: 知能, base: 知能, pos: 名詞, pos1: 一般
surface: （, base: （, pos: 記号, pos1: 括弧開
surface: じん, base: じん, pos: 名詞, pos1: 一般
surface: こうち, base: こうち, pos: 名詞, pos1: 一般
surface: のう, base: のう, pos: 助詞, pos1: 終助詞
surface: 、, base: 、, pos: 記号, pos1: 読点
surface: 、, base: 、, pos: 記号, pos1: 読点
surface: AI, base: *, pos: 名詞, pos1: 一般
surface: 〈, base: 〈, pos: 記号, pos1: 括弧開
surface: エーアイ, base: *, pos: 名詞, pos1: 固有名詞
surface: 〉, base: 〉, pos: 記号, pos1: 括弧閉
surface: ）, base: ）, pos: 記号, pos1: 括弧閉
surface: と, base: と, pos: 助詞, pos1: 格助詞
surface: は, base: は, pos: 助詞, pos1: 係助詞
surface: 、, base: 、, pos: 記号, pos1: 読点
surface: 「, base: 「, pos: 記号, pos1: 括弧開
surface: 『, base: 『, pos: 記号, pos1: 括弧開
surface: 計算, base: 計算, pos: 名詞, pos1: サ変接続
surface: （, base: （, pos: 記号, pos1: 括弧開
surface: ）, base: ）, pos: 記号, pos1: 括弧閉
surface: 』, base: 』, pos: 記号, pos1: 括弧閉
surface: という, base: という, pos: 助詞, pos1: 格助詞
surface: 概念, base: 概念, pos: 名詞, pos1: 一般
surface: と, base: と, pos: 助詞, pos1: 並立助詞
surface: 『, base: 『, po

### 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [30]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [33]:
with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []

for i in range(0, 35):
    print(f'morphs : {"".join([j.surface for j in chunks[i].morphs])}, dst : {chunks[i].dst}, srcs : {chunks[i].srcs}')

morphs : 人工知能, dst : 17, srcs : []
morphs : （じんこうちのう、、, dst : 17, srcs : []
morphs : AI, dst : 3, srcs : []
morphs : 〈エーアイ〉）とは、, dst : 17, srcs : [2]
morphs : 「『計算, dst : 5, srcs : []
morphs : （）』という, dst : 9, srcs : [4]
morphs : 概念と, dst : 9, srcs : []
morphs : 『コンピュータ, dst : 8, srcs : []
morphs : （）』という, dst : 9, srcs : [7]
morphs : 道具を, dst : 10, srcs : [5, 6, 8]
morphs : 用いて, dst : 12, srcs : [9]
morphs : 『知能』を, dst : 12, srcs : []
morphs : 研究する, dst : 13, srcs : [10, 11]
morphs : 計算機科学, dst : 14, srcs : [12]
morphs : （）の, dst : 15, srcs : [13]
morphs : 一分野」を, dst : 16, srcs : [14]
morphs : 指す, dst : 17, srcs : [15]
morphs : 語。, dst : 34, srcs : [0, 1, 3, 16]
morphs : 「言語の, dst : 20, srcs : []
morphs : 理解や, dst : 20, srcs : []
morphs : 推論、, dst : 21, srcs : [18, 19]
morphs : 問題解決などの, dst : 22, srcs : [20]
morphs : 知的行動を, dst : 24, srcs : [21]
morphs : 人間に, dst : 24, srcs : []
morphs : 代わって, dst : 26, srcs : [22, 23]
morphs : コンピューターに, dst : 26, srcs : []
morphs : 行わせる, dst : 27, sr

In [ ]:
'''
保存用
'''
with open('nlp05_ap.txt', encoding='utf-8', mode='w') as f:
    for i in range(len(chunks)):
        f.write(f'morphs : {"".join([j.surface for j in chunks[i].morphs])}, dst : {chunks[i].dst}, srcs : {chunks[i].srcs}\n')

### 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [71]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [99]:
'''
chunksリストの作成
'''

with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []


In [100]:
for i in range(len(chunks)):
    pre = "".join([j.surface for j in chunks[i].morphs])
    point = "".join([j.surface for j in chunks[chunks[i].dst].morphs])
    # 句読点を除去
    pre = pre.replace('、', '').replace('。', '')
    point = point.replace('、', '').replace('。', '')
    print(f'{pre}\t{point}')

人工知能	語
（じんこうちのう	語
AI	〈エーアイ〉）とは
〈エーアイ〉）とは	語
「『計算	（）』という
（）』という	道具を
概念と	道具を
『コンピュータ	（）』という
（）』という	道具を
道具を	用いて
用いて	研究する
『知能』を	研究する
研究する	計算機科学
計算機科学	（）の
（）の	一分野」を
一分野」を	指す
指す	語
語	研究分野」とも
「言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術」または
技術」または	研究分野」とも
「計算機	（コンピュータ）による
（コンピュータ）による	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野」とも
研究分野」とも	される
される	つかないでしょうね」と
『日本大百科全書(ニッポニカ)』の	（じんこうちのう
解説で	（）』という
情報工学者・通信工学者の	〈エーアイ〉）とは
佐藤理史は	（）』という
次のように	（）』という
述べている	つかないでしょうね」と
人間の	（じんこうちのう
知的能力を	〈エーアイ〉）とは
コンピュータ上で	〈エーアイ〉）とは
実現する	（）』という
様々な	（）』という
技術・ソフトウェア・コンピュータシステム	理解や
応用例は	理解や
自然言語処理	（）』という
（機械翻訳・かな漢字変換・構文解析等）	道具を
専門家の	用いて
推論・判断を	『知能』を
模倣する	研究する
エキスパートシステム	「言語の
画像データを	（）の
解析して	語
特定の	指す
パターンを	語
検出・抽出したりする	「言語の
画像認識等が	理解や
ある	人間に
1956年に	人間に
ダートマス会議で	人間に
ジョン・マッカーシーにより	人間に
命名された	される
現在では	される
記号処理を	行わせる
用いた	技術」または
知能の	「計算機
記述を	知的な
主体と	知的な
する	設計や
情報処理や	設計や
研究での	実現に関する
アプローチという	研究分野」とも
意味あいでも	される
使われている	知的能力を
家庭用電気機械器具の	解説で
制御システムや	佐藤理史は
ゲームソフトの	佐藤理史

### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [2]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [3]:
'''
chunksリストの作成
'''

with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []


In [16]:
for chunk in chunks:
    x = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号" and morph.pos == "名詞"])
    y = "".join([morph.surface for morph in chunks[chunk.dst].morphs if morph.pos != "記号" and morph.pos == "動詞"])
    if x != "" and y != "":
        print(f'{x}\t{y}')

道具	用い
知能	する
一分野	指す
知的行動	代わっ
人間	代わっ
コンピューター	行わせる
研究分野	される
専門家	用い
模倣	する
特定	指す
命名	される
現在	される
記号処理	行わせる
意味あい	される
思考ルーチン	述べいる
専門家	する
記述	代わっ
問題	代わっ
実用	行わせる
アプローチ	される
アプローチ	される
記述	述べいる
知性	ある
強力	ある
実装	用い
作業	つか
流行	用い
社会	する
トップ棋士	行わせる
不完全情報ゲーム	代わっ
世界トップクラス	行わせる
時代	される
CI	する
パラメータ調整	する
コネクショニズム	する
学習	指す
関係	行わせる
手法	行わせる
もの	行わせる
ニューラルネットワーク	する
生成	つか
最近	行わせる
派生	代わっ
分野	行わせる
ディープラーニング	される
分野	される
成果	される
これら	する
コンテンツ生成	する
指摘	つか
チャールズバベッジ	指す
ウォルター・ピッツ	指す
神経活動	用い
アイデア	する
論文	指す
1943年	指す
ジョンマッカーシー	する
会議	する
用語	する
導入	代わっ
ジョセフ・ワイゼンバウム	代わっ
おしゃべりロボット	つか
使用	用い
分野	用い
創立	つか
マービン・ミンスキー	用い
シーモア・パパート	指す
パーセプトロン	する
出版	指す
限界	指す
テッド・ショートリッフェ	代わっ
ルールベースシステム	代わっ
知識表現	行わせる
障害物	つか
開発	つか
よう	つか
提唱	つか
ボードゲーム	用い
1992年	指す
IBM	指す
バックギャモン専用コンピュータTDギャモン	指す
同年	代わっ
湾岸戦争	される
ユニット	される
コスト	する
投資全額	する
優秀	する
論理	し
指摘	つか
開始	代わっ
セマンティック・ウェブ	代わっ
規格	する
知識工学	述べいる
OWL	する
前半	し
規格化	し
完了	され
Web開発者	され
開発工数	しする
メリット	ある
こと	され
現在	され
普及	され
計算リソース	用い
不足	する
産業	指す
終焉	つか
エキスパートシステム	する
専門	する
採用	用い
膨大	する
等	代わっ
PID制御	用い
問題	する
知的制御	指す
盛ん	指す
学習	代わっ
個数	述べいる
多様	する


### 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [3]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [4]:
'''
chunksリストの作成
'''

with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []


In [6]:
from graphviz import Digraph

graph = Digraph(format='png')
graph.attr('node', fontname='MS Gothic')
for i in range(len(chunks)):
    if chunks[i].dst >= 0:
        x = "".join([i.surface for i in chunks[i].morphs])
        y = "".join([i.surface for i in chunks[chunks[i].dst].morphs])
        graph.node(x)
        graph.node(y)
        graph.edge(x, y)
graph.render('image44')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.327936 to fit


'image44.png'

### 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

```
作り出す	で は を
```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語と格パターンの組み合わせ
「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [17]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [18]:
'''
chunksリストの作成
'''

with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []

In [28]:
with open('text45.txt', encoding='utf-8', mode='w') as f:
    for chunk in chunks:
        x = "".join([morph.base for morph in chunk.morphs if morph.pos == "動詞"])
        if x == '':
            continue
        if len(chunk.srcs) == 1:
            y = "".join([morph.base for morph in chunks[chunk.srcs[0]].morphs if morph.pos != "記号" and morph.pos == "助詞"])
            if y == '':
                continue
            f.write(f'{x}\t{y}\n')
            #print(f'{x}\t{y}\n')
        else:
            y = ''
            for src in chunk.srcs:
                #y = " ".join([morph.base for morph in chunks[src].morphs if morph.pos != "記号" and morph.pos == "助詞"])
                if x == '作り出す':
                        print(''.join([morph.surface for morph in chunks[src].morphs]))
                for morph in chunks[src].morphs:
                    if morph.pos != "記号" and morph.pos == "助詞":
                        y += f'{morph.base} '
            f.write(f'{x} {y}\n')
            #print(f'{x} {y}\n')

用いて
『知能』を
模倣する
専門家の
超えて
社会に
（CI）は
（例えば、
パラメータ調整、
コネクショニズムの
ニューラルネットワークや
アイデアの
出始めた。
ジョン・マッカーシーは
会議で
用語を


#### UNIXコマンド
 sort text45.txt | uniq -c | sort -rn | awk -F'[       ]' '{print $2}' | grep -E -x  '行う|なる|与える'<br>
sortでソートをしてuniqでちょうふくをカウントしたのちさらにソートを行う（-nが数値順，-rが逆順）

#### 実行結果
     16 し      を
     14 し      の
     11 する    を
      9 し ね と
      8 する    の
      7 し      て
      6 しいる  を
      6 されいる ね と
      5 する    や
      5 しいる  の

### 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

```
作り出す	で は を	会議で ジョンマッカーシーは 用語を
```

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [2]:
'''
chunksリストの作成
'''

with open("ai.ja.txt.parsed", encoding='utf-8') as f:
    lines = f.read().split('\n')

chunks = []
k=1 # 文節の先頭要素の添え字
for i in range(1, len(lines)):
    if lines[i] == "" or lines[i] == 'EOS':
        continue
    if lines[i][0] == '*':
        _, src, dst, _, _ = lines[k-1].split(' ') # 文節情報から必要な情報のみを抽出

        # chunksを線形探索し，係り元インデックス番号のリストを作成
        srcs = [j for j in range(len(chunks)) if int(src) == chunks[j].dst]
        
        # 文節内の形態要素のリストを作成
        morphs = []
        for j in range(k, i):
            if lines[j] == "" or lines[j] == 'EOS':
                continue
            sur, keys = lines[j].split("\t")
            key = keys.split(",")
            morphs.append(Morph(sur, key[6], key[0], key[1]))

        # 文節をリストに追加
        chunks.append(Chunk(morphs, int(dst[:-1]), srcs))
        k=i+1
        srcs = []

In [ ]:
for chunk in chunks:
    x = "".join([morph.surface for morph in chunk.morphs if morph.pos == "動詞"])
    if x == '':
        continue
    count = 0
    for morph in chunks[chunk.dst].morphs:
        if morph.pos == '助詞':
            count += 1
    if count == 1:
        y = "".join([morph.surface for morph in chunks[chunk.dst].morphs if morph.pos != "記号" and morph.pos == "助詞"])
        if y == '':
            continue
        y_ = "".join([morph.surface for morph in chunks[chunk.dst].morphs if morph.pos != "記号" and morph.pos == "助詞"])
        print(f'{x}\t{y}')
    else:
        y = " ".join([morph.surface for morph in chunks[chunk.dst].morphs if morph.pos != "記号" and morph.pos == "助詞"])
        if y == '':
            continue
        print(f'{x} {y}')